In [11]:
import pandas as pd
import datetime as dt

In [16]:
df_ = pd.read_excel("datasets/online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")

In [48]:
df = df_.copy()

In [49]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [50]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [51]:
df = df[~df["Invoice"].str.contains("C", na=False)]

In [52]:
df = df[df["Quantity"] > 0]

In [53]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [54]:
df.isnull().sum()

Invoice             0
StockCode           0
Description       592
Quantity            0
InvoiceDate         0
Price               0
Customer ID    133361
Country             0
TotalPrice          0
dtype: int64

In [55]:
df.dropna(inplace=True)

In [56]:
df.describe([0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,397925.0,13.021793,180.419984,1.0,1.00,1.00,1.00,2.00,6.00,12.00,24.00,36.0,120.00,80995.00
Price,397925.0,3.116212,22.096773,0.0,0.21,0.42,0.55,1.25,1.95,3.75,6.35,8.5,14.95,8142.75
Customer ID,397925.0,15294.308601,1713.172738,12346.0,12415.00,12627.00,12883.00,13969.00,15159.00,16795.00,17725.00,17912.0,18211.00,18287.00
TotalPrice,397925.0,22.394737,309.055200,0.0,0.55,1.25,1.95,4.68,11.80,19.80,35.40,67.5,202.50,168469.60


In [57]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
TotalPrice     0
dtype: int64

In [58]:
df["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [59]:
today_date = dt.datetime(2011, 12, 11)

In [60]:
rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda date: (today_date - date.max()).days,
                                     "Invoice": lambda amount: len(amount),
                                     "TotalPrice": lambda total_price: total_price.sum()})

In [61]:
rfm.columns = ["Recency", "Frequency", "Monetary"]

In [62]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,326,1,77183.60
12347.0,3,182,4310.00
12348.0,76,31,1797.24
12349.0,19,73,1757.55
12350.0,311,17,334.40


In [63]:
rfm.shape

(4339, 3)

In [64]:
rfm = rfm[(rfm["Monetary"]) > 0 & (rfm["Frequency"] > 0)]

In [65]:
rfm.shape

(4338, 3)

In [66]:
rfm["RecencyScore"] = pd.qcut(rfm["Recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm["Frequency"], 5, labels=[1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm["Monetary"], 5, labels=[1, 2, 3, 4, 5])

In [67]:
rfm["RFM_Scores"] = (rfm["RecencyScore"].astype(str) +
                     rfm["FrequencyScore"].astype(str) +
                     rfm["MonetaryScore"].astype(str))

In [68]:
rfm[rfm["RFM_Scores"] == "555"]

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Scores
Customer ID,,,,,,,
12347.0,3,182,4310.00,5,5,5,555
12362.0,4,266,5226.23,5,5,5,555
12417.0,4,192,3649.10,5,5,5,555
12433.0,1,420,13375.87,5,5,5,555
12437.0,2,200,4951.41,5,5,5,555
...,...,...,...,...,...,...,...
18225.0,4,271,5509.12,5,5,5,555
18229.0,12,164,7276.90,5,5,5,555
18245.0,8,175,2567.06,5,5,5,555


In [38]:
seg_map = {
    r"[1-2][1-2]": "Hibernating",
    r"[1-2][3-4]": "At_Risk",
    r"[1-2]5": "Cant_Lose_Them",
    r"3[1-2]": "About_to_Sleep",
    r"33": "Need_Attention",
    r"[3-4][4-5]": "Loyal_Customer",
    r"41": "Promising",
    r"[4-5][2-3]": "Potential_Loyalists",
    r"51": "New_Customer",
    r"5[4-5]": "Champions"
}

In [39]:
rfm["Segment"] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)

In [40]:
rfm["Segment"] = rfm["Segment"].replace(seg_map, regex=True)

In [41]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_Scores,Segment
Customer ID,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,Hibernating
12347.0,3,182,4310.00,5,5,5,555,Champions
12348.0,76,31,1797.24,2,3,4,234,At_Risk
12349.0,19,73,1757.55,4,4,4,444,Loyal_Customer
12350.0,311,17,334.40,1,2,2,122,Hibernating


In [42]:
df["Customer ID"].nunique()

4339

In [43]:
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count"])

Recency         Frequency           Monetary      
                           mean count        mean count         mean count
Segment                                                                   
About_to_Sleep        53.192547   322   15.987578   322   450.997174   322
At_Risk              166.435852   569   56.859402   569   996.916872   569
Cant_Lose_Them       144.226190    84  181.666667    84  2370.705012    84
Champions              6.400990   606  289.031353   606  6960.915446   606
Hibernating          210.251397  1074   13.589385  1074   536.535672  1074
Loyal_Customer        34.206854   817  157.116279   817  2845.732852   817
Need_Attention        52.985366   205   41.736585   205   856.195854   205
New_Customer           7.421053    57    7.578947    57  3618.697018    57
Potential_Loyalists   16.668699   492   34.945122   492   915.486994   492
Promising             23.437500   112    7.767857   112   429.433929   112

In [44]:
new_df = pd.DataFrame()

In [45]:
new_df["Loyal_Customer"] = rfm[rfm["Segment"] == "Loyal_Customer"].index

In [46]:
new_df.to_excel("Loyal_Customer.xlsx")